学习深度学习计算。

# 层和块

In [5]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1560, -0.0823, -0.0718,  0.0643,  0.1892,  0.2472,  0.0082,  0.0526,
         -0.0343, -0.1716],
        [-0.2018, -0.1450, -0.0185,  0.1079,  0.2328,  0.4292,  0.0401, -0.0474,
         -0.0655, -0.1958]], grad_fn=<AddmmBackward0>)

In [6]:
from torch import nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

In [7]:
net = MLP()
net(X)

# tensor([[ 0.1912,  0.0413,  0.0140,  0.0509, -0.1169, -0.0441,  0.0614,  0.0376,
#          -0.0812, -0.1776],
#         [ 0.0929, -0.0536, -0.0730,  0.0093, -0.1146, -0.0210,  0.0392, -0.0453,
#          -0.1113, -0.1817]], grad_fn=<AddmmBackward0>)

tensor([[-0.0898,  0.0304,  0.0319, -0.0925,  0.1536, -0.1776,  0.2120,  0.1081,
          0.0109, -0.2248],
        [-0.0821,  0.0304, -0.0064, -0.1893,  0.1903, -0.1745,  0.1479,  0.0573,
         -0.0403, -0.2334]], grad_fn=<AddmmBackward0>)

In [8]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [9]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

# tensor([[-0.1229, -0.3086, -0.0750,  0.0414,  0.1376, -0.0072, -0.0623,  0.1302,
#           0.0567, -0.1040],
#         [-0.0971, -0.2352, -0.0374,  0.0582,  0.1482, -0.1153, -0.1219,  0.0907,
#          -0.0910, -0.0503]], grad_fn=<AddmmBackward0>)

tensor([[ 0.0188, -0.1178,  0.0245,  0.1149,  0.0011, -0.2958,  0.1410, -0.1120,
          0.3150, -0.0518],
        [ 0.1617, -0.1229,  0.1111,  0.1013,  0.0314, -0.2863,  0.1674, -0.0955,
          0.2517, -0.1605]], grad_fn=<AddmmBackward0>)

In [10]:
from torch import nn
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.norm() > 1:
            X /= 2
        return X

In [11]:
import torch
net = FixedHiddenMLP()
net(X)

# tensor([[-3.5930e-02, -6.9781e-03, -9.4119e-02,  1.1582e-01,  2.3377e-02,
#          -6.6372e-02, -3.2058e-02, -1.2786e-02, -2.3962e-02, -3.1722e-02,
#          -1.8784e-01, -9.4819e-05,  2.0893e-01,  2.1778e-01,  1.4351e-01,
#          -2.6391e-02,  6.5536e-03,  2.2466e-01, -9.9925e-02,  1.7696e-01],
#         [-5.0210e-02, -5.7538e-03, -8.4254e-02,  1.0451e-01,  2.4118e-02,
#          -4.9907e-02, -1.8658e-02, -3.4405e-02, -2.3964e-02, -5.3667e-02,
#          -1.9062e-01,  7.6478e-03,  2.0308e-01,  2.3180e-01,  1.2236e-01,
#          -2.7363e-02,  1.1634e-03,  2.2023e-01, -1.0947e-01,  1.9209e-01]],
#        grad_fn=<DivBackward0>)

tensor([[-0.0137, -0.0107, -0.1359, -0.1292, -0.1642,  0.1579, -0.0298, -0.0687,
         -0.0414,  0.0383,  0.0021,  0.1122,  0.1034,  0.0460,  0.0641,  0.1318,
          0.0913, -0.0712,  0.0631,  0.0870],
        [-0.0264,  0.0202, -0.1371, -0.1727, -0.2293,  0.1529, -0.0139, -0.1404,
         -0.0700,  0.0499,  0.0065,  0.1523,  0.1435,  0.0906,  0.0278,  0.1299,
          0.1303, -0.1215,  0.0830,  0.1561]], grad_fn=<DivBackward0>)

In [12]:
from torch import nn
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

# tensor([[ 0.0388,  0.0290, -0.0526,  0.0514, -0.0501, -0.1435, -0.1622, -0.1392,
#          -0.1977,  0.0007,  0.0417, -0.0216, -0.0071,  0.0533,  0.0229,  0.0953,
#          -0.0976, -0.0235,  0.0743, -0.0100],
#         [ 0.0376,  0.0295, -0.0516,  0.0523, -0.0497, -0.1445, -0.1619, -0.1384,
#          -0.1969, -0.0015,  0.0407, -0.0220, -0.0069,  0.0526,  0.0213,  0.0954,
#          -0.0972, -0.0245,  0.0751, -0.0102]], grad_fn=<DivBackward0>)

tensor([[ 0.0388,  0.0290, -0.0526,  0.0514, -0.0501, -0.1435, -0.1622, -0.1392,
         -0.1977,  0.0007,  0.0417, -0.0216, -0.0071,  0.0533,  0.0229,  0.0953,
         -0.0976, -0.0235,  0.0743, -0.0100],
        [ 0.0376,  0.0295, -0.0516,  0.0523, -0.0497, -0.1445, -0.1619, -0.1384,
         -0.1969, -0.0015,  0.0407, -0.0220, -0.0069,  0.0526,  0.0213,  0.0954,
         -0.0972, -0.0245,  0.0751, -0.0102]], grad_fn=<DivBackward0>)

# 参数管理

In [13]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

# tensor([[-0.1822],
#         [-0.3126]], grad_fn=<AddmmBackward0>)

tensor([[-0.1822],
        [-0.3126]], grad_fn=<AddmmBackward0>)

In [15]:
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

In [17]:
net(X)

tensor([[ 0.1093],
        [-0.1140]], grad_fn=<AddmmBackward0>)

In [19]:
net[2].state_dict()

# OrderedDict([('weight',
#               tensor([[-0.3444,  0.3052, -0.3438, -0.1365,  0.2980, -0.2586, -0.1679, -0.1135]])),
#              ('bias', tensor([-0.2424]))])

OrderedDict([('weight',
              tensor([[-0.3444,  0.3052, -0.3438, -0.1365,  0.2980, -0.2586, -0.1679, -0.1135]])),
             ('bias', tensor([-0.2424]))])

In [20]:
net[1].state_dict()

OrderedDict()

In [21]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

# <class 'torch.nn.parameter.Parameter'>
# Parameter containing:
# tensor([-0.2424], requires_grad=True)
# tensor([-0.2424])

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2424], requires_grad=True)
tensor([-0.2424])


In [22]:
net[2].weight.grad == None

# True

True

In [23]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

# ('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
# ('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [24]:
net.state_dict()['2.bias'].data

# tensor([-0.2424])

tensor([-0.2424])

In [25]:
def block1():
    block = nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
    )

    return block

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f"block {i}", block1())
    return net

rgnet = nn.Sequential(
    block2(),
    nn.Linear(4, 1)
)

rgnet(X)

# tensor([[-0.3489],
#         [-0.3489]], grad_fn=<AddmmBackward0>)

tensor([[-0.3489],
        [-0.3489]], grad_fn=<AddmmBackward0>)

In [27]:
print(rgnet)

# Sequential(
#   (0): Sequential(
#     (block 0): Sequential(
#       (0): Linear(in_features=4, out_features=8, bias=True)
#       (1): ReLU()
#       (2): Linear(in_features=8, out_features=4, bias=True)
#       (3): ReLU()
#     )
#     (block 1): Sequential(
#       (0): Linear(in_features=4, out_features=8, bias=True)
#       (1): ReLU()
#       (2): Linear(in_features=8, out_features=4, bias=True)
#       (3): ReLU()
#     )
#     (block 2): Sequential(
#       (0): Linear(in_features=4, out_features=8, bias=True)
#       (1): ReLU()
#       (2): Linear(in_features=8, out_features=4, bias=True)
#       (3): ReLU()
#     )
#     (block 3): Sequential(
#       (0): Linear(in_features=4, out_features=8, bias=True)
#       (1): ReLU()
#       (2): Linear(in_features=8, out_features=4, bias=True)
#       (3): ReLU()
#     )
#   )
#   (1): Linear(in_features=4, out_features=1, bias=True)
# )

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [29]:
rgnet[0][1][0].bias.data

# tensor([ 0.2957,  0.2749,  0.1099, -0.2194,  0.4637, -0.1263, -0.3124,  0.3046])

tensor([ 0.2957,  0.2749,  0.1099, -0.2194,  0.4637, -0.1263, -0.3124,  0.3046])

In [30]:
rgnet[1].weight.data

tensor([[-0.0068,  0.3340, -0.0026, -0.2887]])

In [34]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

# (tensor([ 0.0028, -0.0031,  0.0210, -0.0002]), tensor(0.))

(tensor([ 0.0028, -0.0031,  0.0210, -0.0002]), tensor(0.))

In [35]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

# (tensor([1., 1., 1., 1.]), tensor(0.))

(tensor([1., 1., 1., 1.]), tensor(0.))

In [36]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

# tensor([ 0.3365,  0.4642, -0.1247, -0.1969])
# tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])

tensor([ 0.3365,  0.4642, -0.1247, -0.1969])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [37]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

# tensor([[-0.0000, -0.0000, -0.0000, -0.0000],
#         [ 0.0000,  7.1099, -7.0694, -6.1635]], grad_fn=<SliceBackward0>)

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000, -0.0000, -0.0000],
        [ 0.0000,  7.1099, -7.0694, -6.1635]], grad_fn=<SliceBackward0>)

In [38]:
net[0].weight.data.shape

torch.Size([8, 4])

In [39]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

# tensor([42.,  1.,  1.,  1.])

tensor([42.,  1.,  1.,  1.])

In [40]:
net[0].weight.data

tensor([[42.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000,  8.1099, -6.0694, -5.1635],
        [ 1.0000,  1.0000,  1.0000,  8.5654],
        [ 1.0000, -8.3642,  8.0938,  1.0000],
        [ 9.6432,  1.0000,  1.0000,  1.0000],
        [-8.4329,  1.0000,  1.0000,  1.0000],
        [ 1.0000,  6.0261,  1.0000, 10.3409],
        [ 8.1057,  8.2111,  1.0000,  1.0000]])

In [41]:
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

In [42]:
# 定义共享层
shared = nn.Linear(8, 8)

# 定义网络
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)

net(X)

# tensor([[-0.2248],
#         [-0.2136]], grad_fn=<AddmmBackward0>)

tensor([[-0.2248],
        [-0.2136]], grad_fn=<AddmmBackward0>)

In [43]:
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

# tensor([True, True, True, True, True, True, True, True])
# tensor([True, True, True, True, True, True, True, True])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 自定义层

In [44]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [45]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

# tensor([-2., -1.,  0.,  1.,  2.])

tensor([-2., -1.,  0.,  1.,  2.])

In [46]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [47]:
Y = net(torch.rand(4, 8))
Y.mean()

# tensor(9.3132e-09, grad_fn=<MeanBackward0>)

tensor(9.3132e-09, grad_fn=<MeanBackward0>)

In [64]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units, ))

    def forward(self, X):
        linear = torch.matmul(X, self.weight) + self.bias
        return F.relu(linear)

In [60]:
torch.randn(3, )

tensor([-0.6056,  1.6645, -0.2851])

In [56]:
torch.randn(5, 3)

tensor([[ 0.8582, -0.8524,  1.5880],
        [ 0.0292, -0.0751,  0.9909],
        [-1.7602,  0.3408, -0.6826],
        [-2.7443,  0.0341, -0.6046],
        [ 0.5380, -0.4133, -0.7976]])

In [68]:
linear = MyLinear(5, 3)
linear.weight

# Parameter containing:
# tensor([[-1.2373,  0.0189,  0.2378],
#         [ 1.3135, -1.1337,  0.1082],
#         [ 0.1025,  0.5644,  0.0697],
#         [ 0.8477, -1.5213, -1.0938],
#         [-1.7703, -1.2536, -0.9810]], requires_grad=True)

Parameter containing:
tensor([[-1.2373,  0.0189,  0.2378],
        [ 1.3135, -1.1337,  0.1082],
        [ 0.1025,  0.5644,  0.0697],
        [ 0.8477, -1.5213, -1.0938],
        [-1.7703, -1.2536, -0.9810]], requires_grad=True)

In [69]:
linear(torch.rand(2, 5))

# tensor([[0.0000, 0.0000, 0.0000],
#         [0.0995, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)

tensor([[0.0000, 0.0000, 0.0000],
        [0.0995, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)

In [70]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

# tensor([[16.0757],
#         [ 3.3368]], grad_fn=<ReluBackward0>)

tensor([[16.0757],
        [ 3.3368]], grad_fn=<ReluBackward0>)

# 读写文件

In [72]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [ ]:
x2 = torch.load('x-file')
x2

# tensor([0, 1, 2, 3])

tensor([0, 1, 2, 3])

In [ ]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

# (tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [ ]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

# {'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [77]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [78]:
torch.save(net.state_dict(), 'mlp.params')

In [ ]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

# MLP(
#   (hidden): Linear(in_features=20, out_features=256, bias=True)
#   (output): Linear(in_features=256, out_features=10, bias=True)
# )

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
Y_clone = clone(X)
Y_clone == Y

# tensor([[True, True, True, True, True, True, True, True, True, True],
#         [True, True, True, True, True, True, True, True, True, True]])

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])